<a href="https://colab.research.google.com/github/JinFree/Thermo-and-Fluid-Engineering-Lab.1/blob/master/2020/Numba.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Numba

## First Steps with numba

In [0]:
import numba
print(numba.__version__)

### Numba
  


* numba는 Python 코드의 특정 부분을 컴파일할 수 있게 하며, 그를 통해 해당 부분이 C 언어로 컴파일된 코드와 같은 속도로 작동할 수 있게 한다.
* numba는 함수 수순에서 작동한다.
* 이 노트북은 numba의 간단한 사용법이 설명되어 있으며, 아래 깃헙 저장소에서 소개된 내용을 한국어로 번역한 것이다.
  * https://github.com/numba/numba

* 간단한 예제
 * Numpy array를 활용하여 Python으로 버블소트를 구현한 경우

In [0]:
def bubblesort(X):
    N = len(X)
    for end in range(N, 1, -1):
        for i in range(end - 1):
            cur = X[i]
            if cur > X[i + 1]:
                tmp = X[i]
                X[i] = X[i + 1]
                X[i + 1] = tmp

* 우선 Numpy array를 생성한 후 랜덤하게 섞는다.

In [0]:
import numpy as np

original = np.arange(0.0, 10.0, 0.01, dtype='f4')
shuffled = original.copy()
np.random.shuffle(shuffled)

* 이후 위에서 구현한 정렬 함수를 사용한다.

In [0]:
sorted = shuffled.copy()
bubblesort(sorted)
print(np.array_equal(sorted, original))

* 이후 수행시간을 확인한다.

In [0]:
sorted[:] = shuffled[:]
%timeit sorted[:] = shuffled[:]; bubblesort(sorted)

In [0]:
%timeit sorted[:] = shuffled[:]

### numba.jit을 활용한 함수의 컴파일



* numba를 활용하는 방법을 소개한다.
* numba.jit 데코레이터를 명시적으로 활용하는 방법이 있다.
* 이를 활용한 후 속도 비교를 수행한다.

In [0]:
print(numba.jit.__doc__)

In [0]:
bubblesort_jit = numba.jit("void(f4[:])")(bubblesort)

* 위 코드를 수행하면, bubblesort_jit는 위에서 선언횐 bubblesort를 컴파일한 함수가 된다.
* "void(f4[:])"는 Signature라 불리며 함수의 입력과 리턴을 의미한다.
* f4[:]는 입력, void는 리턴을 의미한다.

* 작동하는지를 확인한다.

In [0]:
sorted[:] = shuffled[:] # reset to shuffled before sorting
bubblesort_jit(sorted)
print(np.array_equal(sorted, original))

* 이제 작동시간을 확인한다.

In [0]:
%timeit sorted[:] = shuffled[:]; bubblesort_jit(sorted)

In [0]:
%timeit sorted[:] = shuffled[:]; bubblesort(sorted)

* 아래와 같은 방식으로도 함수를 컴파일할 수 있다.

In [0]:
@numba.jit("void(f4[:])")
def bubblesort_jit_function(X):
    N = len(X)
    for end in range(N, 1, -1):
        for i in range(end - 1):
            cur = X[i]
            if cur > X[i + 1]:
                tmp = X[i]
                X[i] = X[i + 1]
                X[i + 1] = tmp

In [0]:
sorted[:] = shuffled[:] # reset to shuffled before sorting
bubblesort_jit_function(sorted)
print(np.array_equal(sorted, original))

### Signature
* Python함수를 컴파일할 때 컴파일러는 함수의 입출력 자료형 정보를 받아야만 한다.
* numba.jit을 통해 컴파일된 함수는 알맞은 입력 자료형에 대해서 정상작동한다. 
 * 그렇지 않은 경우도 가능하나, 흔하지는 않다.
* Signature에는 출력 자료형도 포함되며, 아래와 같은 형태를 가진다.
* <출력 자료형>(<첫 번째 입력 자료형>, <두 번째 입력 자료형>, ...)

### Signature가 제공되지 않은 함수의 컴파일
* numba 버전 0.12 이상에서는 numba.jit에 자료형에 대한 Signature가 제공되지 않아도 컴파일이 가능하다.
* 기존에는 이를 numba.autojit로 제공했으나 최근 버전에서는 numba.jit에 포함되었다.

In [0]:
bubblesort_autojit = numba.jit(bubblesort)

In [0]:
%timeit sorted[:] = shuffled[:]; bubblesort_autojit(sorted)

### 몇가지 추가 사항
* 컴파일에는 시간이 필요하다. 작은 함수를 사용하는 경우에 많은 시간이 걸리지는 않을 것이다. 그러나 많은 기능이 있는 함수를 컴파일 할 때는 시간이 오래 걸릴 수 있다. numba는 가능한 한 컴파일을 빠르게 하기 위해 사용하지 않는 함수는 컴파일을 하지 않는다.
* numba를 활용하는 컴파일보다 Python 네이티브 시스템이 더 빠른 경우가 있다. 이 경우에는 컴파일이 되지 않는다.
* 기본적으로, 'cpu' 타겟은 'nopython' 모드로 컴파일한다. 이것이 실패하면 Python 네이티브 시스템으로 사용한다.

In [0]:
@numba.jit("void(i1[:])")
def test(value):
    for i in range(len(value)):
        value[i] = i % 100

from decimal import Decimal
@numba.jit("void(i1[:])")
def test2(value):
    for i in range(len(value)):
        value[i] = i % Decimal(100)

res = np.zeros((10000,), dtype="i1")

In [0]:
%timeit test(res)

In [0]:
%timeit test2(res)

* 'nopython'모드의 코드 컴파일이 실패하는 경우 피드백을 받는 방법은 아래와 같다.
* nopython=True를 통해 강제적으로 nopython모드의 컴파일을 수행하게 함으로써 에러를 받아볼 수 있다.

* 아래 코드는 정상적으로 numba.jit의 'nopython'모드 컴파일이 수행된다.

In [0]:
@numba.jit("void(i1[:])", nopython=True)
def test(value):
    for i in range(len(value)):
        value[i] = i % 100

* 반면에, 아래 코드는 Decimal사용으로 인해 'nopython'모드 컴파일 수행이 불가능하다.
 * 에러가 나오는 것이 정상이다.

In [0]:
@numba.jit("void(i1[:])", nopython=True)
def test2(value):
    for i in range(len(value)):
        value[i] = i % Decimal(100)

## CPU를 활용하는 Numba 병렬 연산

* 아래의 코드는 numpy의 ufunc를 numba를 활용하여 정의하는 코드이다.
* 각 배열 요소에 대해 CPU의 모든 코어를 이용하여 연산을 수행한다.

In [0]:
import math
import numba

@numba.vectorize([numba.float32(numba.float32, numba.float32)], target='parallel')
def func1(x, y):
    return math.atan2(x, y)

* 위 방법으로 정의된 func1 함수는 아래와 같이 사용할 수 있다.

In [0]:
import numpy

x = numpy.arange(100000000, dtype=numpy.float32)
y = numpy.arange(100000000, dtype=numpy.float32)

z = func1(x, y)

* atan2 함수는 numpy의 ubunc로 이미 구현되어 있으며 아래와 같은 방법으로 사용한다.

In [0]:
import numpy

x = numpy.arange(100000000, dtype=numpy.float32)
y = numpy.arange(100000000, dtype=numpy.float32)

z = numpy.arctan2(x, y)

* func1의 결과와 numpy.atan2의 경과는 동일할 것이며 이를 확인하는 온전한 코드는 아래와 같다.

In [0]:
import math

import numpy
from numba import vectorize, float32

@vectorize([float32(float32, float32)], target='parallel')
def func1(x, y):
    return math.atan2(x, y)


x = numpy.arange(100000000, dtype=numpy.float32)
y = numpy.arange(100000000, dtype=numpy.float32)

z1 = func1(x, y)
z2 = numpy.arctan2(x, y)

assert numpy.all(z1 == z2)

* %timeit을 통해 수행시간을 비교한다.

In [0]:
%timeit z1 = func1(x, y)

In [0]:
%timeit z2 = numpy.arctan2(x, y)

## GPU를 활용하는 Numba 병렬 연산

### CUDA C/C++ vs. Numba vs. pyCUDA

* CUDA C/C++
 * 가장 빠르고 확실한 CUDA 병렬가속화 방법

* pyCUDA
 * CUDA C/C++ API를 활용할 수 있음
 * Python에서 CUDA를 활용할 수 있는 가장 성능이 좋은 활용방법
 * Python에서 C코드를 작성해야 하며, 코드를 많이 바꿔야만 함

* Numba
 * pyCUDA보다 느릴 가능성 있음
 * CUDA C/C++ API를 (아직은) 활용할 수 없음
 * 여전히 Numpy보다는 고속화가 가능함
 * CPU 기반 병렬연산에도 활용할 수 있음
 * Python에서의 OpenACC라고 볼 수 있음

### ufuncs for the GPU

* numba는 @vectorize 데코레이터를 활용하여 ufuncs의 병렬연산을 할 수 있음
* @vectorize만 붙이면 CPU병렬화
* @vectorize([explicit type signature], target='cuda')를 쓰면 GPU병렬화
 * GPU병렬화를 하는 경우 type signature를 맞게 작성해야만 함

* Google Colab에서 Numba CUDA를 활용하기 위해서는 아래 셀을 통해 libdevice 및 libnvvm.so의 위치를 확인한 후 Python 환경에 등록해주어야만 함

In [0]:
!find / -iname 'libdevice'
!find / -iname 'libnvvm.so'

In [0]:
import os
os.environ['NUMBAPRO_LIBDEVICE'] = "/usr/local/cuda-10.0/nvvm/libdevice"
os.environ['NUMBAPRO_NVVM'] = "/usr/local/cuda-10.0/nvvm/lib64/libnvvm.so"

In [0]:
import numpy as np
from numba import vectorize

a = np.array([1, 2, 3, 4])
b = np.array([10, 20, 30, 40])

@vectorize
def add_ufunc_cpu(x, y):
    return x + y # This scalar operation will be performed on each element

@vectorize(['int64(int64, int64)'], target='cuda') # Type signature and target are required for the GPU
def add_ufunc(x, y):
    return x + y

In [0]:
print(add_ufunc_cpu(a, b)) # pass the whole array into the ufunc, it performs the operation on each element
print(add_ufunc(a, b))

In [0]:
%timeit np.add(a, b)   # NumPy on CPU

In [0]:
%timeit add_ufunc_cpu(a, b) # Numba on CPU

In [0]:
%timeit add_ufunc(a, b) # Numba on GPU

* GPU에서의 연산이 CPU보다 느림 이유는 아래의 네가지임
 * 입력 데이터가 충분히 크지 않음: GPU병렬화는 데이터가 클 수록 효과적임
 * 계산이 매우 간단함: CPU에서 수행하는 것이 GPU에 명령어와 데이터를 주고 받는것보다 더 빠를 수 있음
 * 데이터를 GPU로 보내고 받음: 함수의 실행 과정에서 CPU에 선언된 데이터를 GPU로 보내고, 함수의 결과 역시 GPU에서 CPU로 받아옴
 * 필요한 것 이상으로 data type이 큼: GPU 병렬화에 최적화된 data type은 float 32이며, 그보다 큰 데이터 형태는 GPU 병렬화에 장애물임

* 위에 적힌 것을 상기하며, 아래의 코드 뭉치를 실험해볼 수 있음

In [0]:
import math # CUDA target으로 계산을 수행할 때, Numpy의 연산이 아닌 math의 scalar 연산을 사용해야만 함

SQRT_2PI = np.float32((2*math.pi)**0.5)  # Precompute this constant as a float32.  Numba will inline it at compile time.

@vectorize(['float32(float32, float32, float32)'], target='cuda')
def gaussian_pdf(x, mean, sigma):
    '''Compute the value of a Gaussian probability density function at x with given mean and sigma.'''
    return math.exp(-0.5 * ((x - mean) / sigma)**2) / (sigma * SQRT_2PI)

@vectorize
def cpu_gaussian_pdf(x, mean, sigma):
    '''Compute the value of a Gaussian probability density function at x with given mean and sigma.'''
    return math.exp(-0.5 * ((x - mean) / sigma)**2) / (sigma * SQRT_2PI)

import numpy as np
# Evaluate the Gaussian a million times!
x = np.random.uniform(-3, 3, size=1000000).astype(np.float32)
mean = np.float32(0.0)
sigma = np.float32(1.0)

# Quick test on a single element just to make sure it works
gaussian_pdf(x[0], 0.0, 1.0)

import scipy.stats # for definition of gaussian distribution, so we can compare CPU to GPU time
norm_pdf = scipy.stats.norm

In [0]:
%timeit norm_pdf.pdf(x, loc=mean, scale=sigma)

In [0]:
%timeit gaussian_pdf(x, mean, sigma)

In [0]:
%timeit cpu_gaussian_pdf(x, mean, sigma)

* 많이 빨라진 것을 알 수 있으나 더 가속화할 수 있음

### GPU에서 사용 가능한 Python 함수

* if / elif / else
* while / for
* Basic math operators
* math 와 cmath 모듈에 있는 일부 함수
* Tuples
* [the Numba manual](http://numba.pydata.org/numba-doc/latest/cuda/cudapysupported.html)에서 자세한 것을 확인할 수 있음

#### GPU 함수 실습

In [0]:
# This allows us to plot right here in the notebook
%matplotlib inline

# Hacking up a noisy pulse train
import numpy as np
from matplotlib import pyplot as plt

n = 100000
noise = np.random.normal(size=n) * 3
pulses = np.maximum(np.sin(np.arange(n) / (n / 23)) - 0.3, 0.0)
waveform = ((pulses * 300) + noise).astype(np.int16)
plt.plot(waveform)

In [0]:
from numba import vectorize
@vectorize(['int16(int16, int16)'], target='cuda')
def zero_suppress(waveform_value, threshold):
    if waveform_value < threshold:
        result = 0
    else:
        result = waveform_value
    return result

In [0]:
# This will throw an error until you successfully vectorize the `zero_suppress` function above.
# The noise on the baseline should disappear when zero_suppress is implemented
plt.plot(zero_suppress(waveform, 15))

### GPU Memory 제어

* CPU에서 사용하는 메모리와 GPU에서 사용하는 메모리는 서로 다름.
* CPU메모리를 GPU메모리로 복사한 후 GPU 컴파일 된 함수에서 사용할 시 더 빠르게 연산을 수행할 수 있음

In [0]:
@vectorize(['float32(float32, float32)'], target='cuda')
def add_ufunc(x, y):
    return x + y

n = 100000
x = np.arange(n).astype(np.float32)
y = 2 * x

In [0]:
%timeit add_ufunc(x, y)  # Baseline performance with host arrays

* 위에서는 CPU에서 메모리를 바로 GPU컴파일 된 함수에 넣었음.
* 아래 예제에서는 CPU메모리를 GPU메모리로 복사한 수 함수에 입력함

In [0]:
from numba import cuda

x_device = cuda.to_device(x)
y_device = cuda.to_device(y)

print(x_device)
print(x_device.shape)
print(x_device.dtype)

In [0]:
%timeit add_ufunc(x_device, y_device)

* 똑같은 연산을 수행하는데 메모리를 미리 복사해두는지에 따라 절반 가까이 연산시간이 줄어드는 것을 확인할 수 있음

* 결과를 받기 위한 임의의 GPU메모리 어레이를 생성하는 방법은 아래와 같으며, 아래 방법을 통한 생성은 메모리 내부 값이 초기화되지 않음

In [0]:
out_device = cuda.device_array(shape=(n,), dtype=np.float32)  # does not initialize the contents, like np.empty()

* 연산 수행 결과를 받을 메모리를 명시적으로 함수에 넘겨주는 경우 보다 빠르게 함수 수행이 가능해짐

In [0]:
%timeit add_ufunc(x_device, y_device, out=out_device)

* GPU메모리를 CPU메모리로 복사하는 방법은 아래와 같음

In [0]:
out_host = out_device.copy_to_host()
print(out_host[:10])